In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
import os
from tableone import TableOne, load_dataset
import pandas as pd

 
print(tf.__version__)

warnings.filterwarnings("ignore")

# gpus = tf.config.list_physical_devices(device_type='GPU')
# tf.config.set_visible_devices(devices=gpus[1], device_type='GPU')

In [ ]:
seed = 2021
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

In [ ]:
copd_patient = pd.read_csv('Data/COPD_patient.csv')
copd_patient.head()

In [ ]:
invalid_cxr = pd.read_csv('Data/Invalid_cxr.csv')
invalid_cxr

In [ ]:
# demographic = pd.read_csv('Data/demographic.csv')
# import random
# from statistics import mode, StatisticsError

# def most_common(l):
#     if (len(l) == 1):
#         return l[0]
    
#     try:
#         most_common = mode(l)
#     except:
#         most_common = random.choice(l)
    
#     if (most_common == 3): 
#         second_common_list = [x for x in l if x != most_common]
#         if (len(second_common_list) > 0):
#             most_common = mode(second_common_list)
#     return most_common

# new_race_list = []
# for i in demographic['race']:
#     l = list(i)
#     l = [x for x in l if x != ',']
#     new_race_list.append(most_common(l))
    
# demographic.drop('race',axis=1, inplace=True)
# demographic['race'] = new_race_list

# demographic.to_csv('Data/Processed_demo.csv')

In [ ]:
def get_vs_type(vs):
    if (len(vs) > 1):
        return 2
    elif (vs == 'Room air'):
        return 0
    elif (vs == 'HighFlow'):
        return 1
    else:
        return 2
    
def race_group(race):
    if (race > 4):
        return 3
    else:
        return race
    
def get_age_group(age):
    if (age < 40):
        return 0
    elif (age < 60):
        return 1
    elif (age < 80):
        return 2
    else:
        return 3

In [ ]:
subject_ids = []
filename = ['TFrecords/mimic-tf-record{i}.tfrecords'.format(i=i) for i in range(24)]
# filename = ['lateral_tf-record{i}.tfrecords'.format(i=i) for i in range(4)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_ids.append(example.features.feature['subject_id'].int64_list.value[0]) # change subject_id to type int

In [ ]:
train, test = train_test_split(np.unique(subject_ids), test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)

In [ ]:
# extract image, subject_id, study_id from tfrecords to 
demographic = pd.read_csv('Data/patient_age_gender.csv')

study_id_array = []
subject_id_array = []

img_train = []
subject_id_train = []
study_id_train = []
copd_train = []
disease_train = []
vs_train = []
age_train = []
races_train = []
gender_train = []

img_test = []
subject_id_test = []
study_id_test = []
copd_test = []
disease_test = []
vs_test = []
age_test = []
races_test = []
gender_test = []

img_val = []
subject_id_val = []
study_id_val = []
copd_val = []
disease_val = []
vs_val = []
age_val = []
races_val = []
gender_val = []

filename = ['../TFrecords/mimic-tf-record{i}.tfrecords'.format(i=i) for i in range(24)]
# filename = ['lateral_tf-record{i}.tfrecords'.format(i=i) for i in range(4)]
raw_dataset = tf.data.TFRecordDataset(filename)

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 

    subject_id = example.features.feature['subject_id'].int64_list.value[0] # change subject_id to type int
    study_id = example.features.feature['study_id'].int64_list.value[0]
    
    invalid = invalid_cxr.loc[invalid_cxr['icu_subject']==subject_id, 'study_id'].values
    if (len(invalid) > 0 and str(study_id) in invalid[0].split(",")):
        continue
    
    if (subject_id in copd_patient['subject_id'].values):
        label = 1
    else:
        label = 0
                    
    sub_y = []
        
    sub_y.append(1 if example.features.feature['Atelectasis'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Cardiomegaly'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Consolidation'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Edema'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Enlarged Cardiomediastinum'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Fracture'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Lung Lesion'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Lung Opacity'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['No Finding'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pleural Effusion'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pleural Other'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pneumonia'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Pneumothorax'].float_list.value[0] == 1 else 0)
    sub_y.append(1 if example.features.feature['Support Devices'].float_list.value[0] == 1 else 0)
    
    race = race_group(example.features.feature['ethnicity'].int64_list.value[0])
    
    try:
        gender = 0 if demographic.loc[demographic['subject_id'] == subject_id, 'gender'].values[0] == 'F' else 1
    except:
        gender = 0
        
    try:
        age = demographic.loc[demographic['subject_id'] == subject_id, 'anchor_age'].values[0]
    except:
        age = 60
    
    byte_arr = example.features.feature['jpg_bytes'].bytes_list.value[0] # format image stored in Tfrecord
        
    if (subject_id in train):
        img_train.append(byte_arr) # img added to list 
        subject_id_train.append(subject_id) # subject_id add to list
        study_id_train.append(study_id)
        copd_train.append(label)
        gender_train.append(gender)
        age_train.append(age)
        races_train.append(race)
        disease_train.append(sub_y)
    elif (subject_id in val):
        img_val.append(byte_arr) # img added to list 
        subject_id_val.append(subject_id) # subject_id add to list
        study_id_val.append(study_id)
        copd_val.append(label)
        gender_val.append(gender)
        age_val.append(age)
        races_val.append(race)
        disease_val.append(sub_y)
    elif (subject_id in test):
        img_test.append(byte_arr) # img added to list 
        subject_id_test.append(subject_id) # subject_id add to list
        study_id_test.append(study_id)
        copd_test.append(label)
        gender_test.append(gender)
        age_test.append(age)
        races_test.append(race)
        disease_test.append(sub_y)
    else:
        continue
        
    subject_id_array.append(subject_id)    
    study_id_array.append(study_id)

In [ ]:
# store data into a new tfrecord
np.random.seed(seed)

diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

record_file_train = 'copd_train_new.tfrecords' #output file 

with tf.io.TFRecordWriter(record_file_train) as writer: #write info into record_file 
    for i in range(len(img_train)):
        example = tf.train.Example() #create example and append img id and study_id to recrod file 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(img_train[i])  
                
        example.features.feature['subject_id'].int64_list.value.append(subject_id_train[i])
        
        example.features.feature['study_id'].int64_list.value.append(study_id_train[i])

        example.features.feature['COPD'].int64_list.value.append(copd_train[i]) # labels 
        
#         example.features.feature['VS'].int64_list.value.append(vs_train[i])
        
        example.features.feature['race'].int64_list.value.append(races_train[i])
        
        example.features.feature['age'].int64_list.value.append(get_age_group(age_train[i]))
        
        example.features.feature['gender'].int64_list.value.append(gender_train[i])
        
        for j in range(14):
            example.features.feature[diseases[j]].int64_list.value.append(disease_train[i][j])

        # append label to the example
        
        writer.write(example.SerializeToString()) #Write to a record file 
        
record_file_test = 'copd_val_new.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(img_val)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(img_val[i])  
                
        example.features.feature['subject_id'].int64_list.value.append(subject_id_val[i])
        
        example.features.feature['study_id'].int64_list.value.append(study_id_val[i])

        example.features.feature['COPD'].int64_list.value.append(copd_val[i])
        
#         example.features.feature['VS'].int64_list.value.append(vs_val[i])
        
        example.features.feature['race'].int64_list.value.append(races_val[i])
        
        example.features.feature['age'].int64_list.value.append(get_age_group(age_val[i]))
        
        example.features.feature['gender'].int64_list.value.append(gender_val[i])
        
        for j in range(14):
            example.features.feature[diseases[j]].int64_list.value.append(disease_val[i][j])

        # append label to the example
        
        writer.write(example.SerializeToString())

record_file_test = 'copd_test_new.tfrecords'

with tf.io.TFRecordWriter(record_file_test) as writer:
    for i in range(len(img_test)):
        example = tf.train.Example() #default module in TFrecord 
        
        example.features.feature['jpg_bytes'].bytes_list.value.append(img_test[i])  
                
        example.features.feature['subject_id'].int64_list.value.append(subject_id_test[i])
        
        example.features.feature['study_id'].int64_list.value.append(study_id_test[i])

        example.features.feature['COPD'].int64_list.value.append(copd_test[i])
        
#         example.features.feature['VS'].int64_list.value.append(vs_test[i])
        
        example.features.feature['race'].int64_list.value.append(races_test[i])
        
        example.features.feature['age'].int64_list.value.append(get_age_group(age_test[i]))
        
        example.features.feature['gender'].int64_list.value.append(gender_test[i])
        
        for j in range(14):
            example.features.feature[diseases[j]].int64_list.value.append(disease_test[i][j])

        # append label to the example
        
        writer.write(example.SerializeToString())

In [ ]:
diseases = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Enlarged Cardiomediastinum', 'Fracture', 'Lung Lesion', 'Lung Opacity', 'No Finding', 'Pleural Effusion', 'Pleural Other', 'Pneumonia', 'Pneumothorax', 'Support Devices']

Races = ['WHITE', 'BLACK/AFRICAN AMERICAN', 'HISPANIC/LATINO', 'OTHER', 'ASIAN']
Age = ['20-40', '40-60', '60-80', '80+']
Gender = ['Female', 'Male']

In [ ]:
import tensorflow as tf

filename = ['tfrecords/copd_mimic_train.tfrecords', 'tfrecords/copd_mimic_test.tfrecords', 'tfrecords/copd_mimic_val.tfrecords']
raw_dataset = tf.data.TFRecordDataset(filename)
copds = []
ages = []
genders = []
races = []
subject_ids = []
copds_all = [0, 0]
diseases_list = [[] for i in range(14)]

for raw_record in raw_dataset:
    #loads data  from the trecord all info 
    example = tf.train.Example() # subject id and study_id (contains several chest Xrays) 
    example.ParseFromString(raw_record.numpy()) #reads the example 
    
    subject_id = example.features.feature['subject_id'].int64_list.value[0]
    copds_all[example.features.feature['COPD'].int64_list.value[0]] += 1
    if (subject_id in subject_ids):
        continue
    
    copds.append(example.features.feature['COPD'].int64_list.value[0])
    races.append(Races[example.features.feature['race'].int64_list.value[0]])
    ages.append(Age[example.features.feature['age'].int64_list.value[0]])
    genders.append(Gender[example.features.feature['gender'].int64_list.value[0]])
    subject_ids.append(subject_id)
    
    for j in range(14):
        diseases_list[j].append(example.features.feature[diseases[j]].int64_list.value)


In [ ]:
from tableone import TableOne

df = pd.DataFrame({'subject_id':subject_ids, 'gender': genders, 
                              'age':ages, 'race': races,
                              'COPD':copds
                             })

columns = ['gender', 'race', 'age']
categorical = ['gender', 'race', 'age']
groupby = 'COPD'
mytable = TableOne(df, columns=columns, categorical=categorical,
                   groupby=groupby, pval=True)

mytable

In [ ]:
copds_all